In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import uproot

%matplotlib widget

In [5]:
plt.style.use('seaborn')
plt.rcParams['axes.grid'] = True
plt.rcParams['axes.formatter.limits'] = (-3,3)
plt.rcParams['axes.formatter.use_mathtext'] = True
# plt.rcParams['font.size']= 16
# plt.rcParams['axes.labelsize'] = 14
# plt.rcParams['axes.titlesize'] = 14
# plt.rcParams['xtick.labelsize'] = 10
# plt.rcParams['ytick.labelsize'] = 10

In [24]:
def hist1D(ax, df, var, cuts=None, y_err=False, falpha=0.4, normed=False, filled = False, params={}):
    #select next color in colormap for use in line and fill.
    color = next(ax._get_lines.prop_cycler)['color']
    
    if cuts is not None: 
        data = df[var[0]][cuts]
    else:
        data = df[var[0]]
        
    #calculate area under the curve if wanting normalised plot    
    weights = None
    if normed is True:
        entries, edges = np.histogram(
            data,
            range = (var[1][1], var[1][2]),
            bins = var[1][0]        
        )
      
        integral = sum(np.diff(edges)*entries)
        weights = np.ones_like(data)/integral
    
    #Draw histogram
    h = ax.hist(data,
    range = (var[1][1], var[1][2]),
    bins = var[1][0],
    weights=weights,
    histtype = 'step',
    linewidth = 1.5,
    color = color,
    **params
    )
 
    #fill area under histo if set
    if filled is True:
        # brute-force: drawing it again to allow separate alpha(opacity) to be used
        h = ax.hist(data,
            range = (var[1][1], var[1][2]),
            bins = var[1][0],
            weights=weights,
            histtype = 'step',
            linewidth = None,
            fill = True,
            alpha = falpha,
            color = color
            )
        
    #error bars    
    if y_err is True:
        if normed is True: 
            yerr = np.sqrt(entries)/integral
            entries = entries/integral
        else:
            entries, edges = np.histogram(
                data,
                range = (var[1][1], var[1][2]),
                bins = var[1][0]        
            )
            yerr = np.sqrt(entries)
            
        bin_width = edges[1]-edges[0]
        bin_centers = edges[:-1] + bin_width/2
        ax.errorbar(bin_centers, entries, yerr=yerr, color='r', ls='', alpha=0.6, label='err', elinewidth=0.6)
        
    #deal with automatic y-axis scaling funk
    # if normed is True:
    #     #axes = plt.gca()
    #     ax.set_ylim([0,max(entries/integral)*1.1])

    ax.set_xlabel(var[2])
    ax.set_xlim(var[1][1], var[1][2])

    
    return #(entries, edges, yerr)

In [11]:
rootdir = "/w/work3/home/pauln/chansertest/pauln/Pi0_PID_"
fs = "FinalState.root"
tree = 'FINALOUTTREE'

fullcomb_file = rootdir+"fullcomb__/"+fs 
fullcomb_masked_file = rootdir+"fullcomb_masked__/"+fs 
photcomb_file = rootdir+"photcomb__/"+fs 
photcomb_masked_file = rootdir+"photcomb_masked__/"+fs 
 
data = uproot.open(fullcomb_file)[tree]
fullcomb_df = data.pandas.df() 

data = uproot.open(fullcomb_masked_file)[tree]
fullcomb_masked_df = data.pandas.df() 

data = uproot.open(photcomb_file)[tree]
photcomb_df = data.pandas.df() 

data = uproot.open(photcomb_masked_file)[tree]
photcomb_masked_df = data.pandas.df()

# data = uproot.open(new_file)['data']
# new_df = data.pandas.df() 
# print(fullcomb_file)

In [36]:
topos = [
    [fullcomb_df, "Full comb."],
    [fullcomb_masked_df, "Full comb. (masked)"], 
    [photcomb_df, "Photon comb."], 
    [photcomb_masked_df, "Photon comb. (masked)"]
]

bins = 200
distros = [
    ["Pi0IMgg", (bins,0.005,0.2), r"$M_{\gamma\gamma}$ $(GeV/c^{2})$"]
]

In [37]:
fig, ax = plt.subplots(1, 1, figsize=(7,5))

for i, var in enumerate(distros):
    for df,label in topos:
        params = {
            'label': label
        }
        hist1D(ax, df, var, params=params)

    ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …